In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
import os

# Define color function based on updated pH value ranges
def get_ph_color(ph):
    if pd.isna(ph):
        return 'gray', 'No Data'
    elif ph <= 4:
        return '#FF0000', 'Highly Acidic'      # red
    elif ph < 6.5:
        return '#FFFF00', 'Moderately Acidic'  # yellow
    elif ph <= 8.5:
        return '#00FF00', 'Acceptable'         # green
    elif ph < 10:
        return '#00BFFF', 'Moderately Alkaline'  # sky blue
    else:
        return '#8A2BE2', 'Highly Alkaline'     # violet

# Define color function for Dissolved Oxygen (DO)
def get_do_color(do):
    if pd.isna(do):
        return 'gray', 'No Data'
    elif do >= 5:
        return '#00FF00', 'Acceptable'     # green - DO >= 5mg/L
    else:
        return '#FF0000', 'Low Oxygen'     # red - DO < 5mg/L

# Define color function for Biochemical Oxygen Demand (BOD)
def get_bod_color(bod):
    if pd.isna(bod):
        return 'gray', 'No Data'
    elif bod <= 3:
        return '#00FF00', 'Acceptable'     # green - BOD <= 3mg/L
    else:
        return '#FF0000', 'High BOD'       # red - BOD > 3mg/L

def load_data_for_year(year):
    """Load data for a specific year"""
    data_file = f'rivers_with_location_{year}.csv'
    
    if not os.path.exists(data_file):
        print(f"Warning: Data file for year {year} not found!")
        return None
    
    rivers_data = pd.read_csv(data_file)
    
    # Convert relevant columns to numeric
    numeric_columns = ['TEMPERATURE_Min', 'TEMPERATURE_Max', 'DO_Min', 'DO_MAX', 
                       'pH_Min', 'PH_Max', 'CONDUCTIVITY_Min', 'CONDUCTIVITY_Max', 
                       'BOD_Min', 'BOD_Max', 'NITRATE_NITRATE_Min', 'NITRATE_NITRATE_Max',
                       'FECALCOLIFORM_Min', 'FECALCOLIFORM_Max', 'TOTALCOLIFORM_Min', 
                       'TOTALCOLIFORM_Max', 'Latitude', 'Longitude']

    for col in numeric_columns:
        if col in rivers_data.columns:
            rivers_data[col] = pd.to_numeric(rivers_data[col], errors='coerce')
    
    # Drop rows with missing lat/long
    rivers_data = rivers_data.dropna(subset=['Latitude', 'Longitude'])
    
    return rivers_data

def create_river_map(data, year, parameter='pH'):
    """Create a map with river lines colored according to selected parameter values"""
    # Create a map centered on India
    india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)
    
    # Add title with year and parameter
    title_html = f'''
    <h3 align="center" style="font-size:20px"><b>India River {parameter} Map - {year}</b></h3>
    '''
    india_map.get_root().html.add_child(folium.Element(title_html))
    
    # Create a marker cluster for stations
    marker_cluster = MarkerCluster().add_to(india_map)
  
    from sklearn.cluster import DBSCAN
        
    # Extract coordinates
    coords = data[['Latitude', 'Longitude']].values
        
    # Perform DBSCAN clustering
    clustering = DBSCAN(eps=0.1, min_samples=1).fit(coords)
    data['cluster'] = clustering.labels_
    
    # Set the parameter column and color function based on selected parameter
    if parameter == 'pH':
        param_col = 'PH_Max'
        color_func = get_ph_color
    elif parameter == 'DO':
        param_col = 'DO_MAX'
        color_func = get_do_color
    elif parameter == 'BOD':
        param_col = 'BOD_Max'
        color_func = get_bod_color
        
    # Group by clusters
    for cluster_id, stations in data.groupby('cluster'):
        if len(stations) >= 2:
            # Calculate average parameter value for this cluster
            avg_val = stations[param_col].mean()
            color, category = color_func(avg_val)
                
            # Sort by latitude/longitude
            stations_sorted = stations.sort_values(['Latitude', 'Longitude'])
                
            # Create line coordinates
            line_coords = stations_sorted[['Latitude', 'Longitude']].values.tolist()
                
            # Add the line to the map
            folium.PolyLine(
                locations=line_coords,
                color=color,
                weight=5,
                opacity=0.8,
                tooltip=f"Water Body Segment {cluster_id}<br>Average {parameter}: {avg_val:.2f}<br>Category: {category}"
            ).add_to(india_map)
    
    # Add markers for each station
    for idx, row in data.iterrows():
        param_value = row[param_col]
        color, category = color_func(param_value)

        popup_text = f"""
        <b>Station Code:</b> {row['STATION CODE']}<br>
        <b>State:</b> {row['STATE']}<br>
        """
        
        if 'RIVER' in row:
            popup_text += f"<b>River:</b> {row['RIVER']}<br>"
            
        popup_text += f"""
        <b>{parameter}:</b> {f"{param_value:.2f}" if not pd.isna(param_value) else 'N/A'}<br>
        <b>Category:</b> {category}
        """

        try:
            lat, lon = float(row['Latitude']), float(row['Longitude'])

            folium.CircleMarker(
                location=[lat, lon],
                radius=6,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_text, max_width=300)
            ).add_to(marker_cluster)
        except:
            print(f"Skipping row with invalid coordinates: {row['STATION CODE']}")

    # Add appropriate legend based on parameter
    if parameter == 'pH':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; 
                    padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <h4>pH Levels</h4>
        <div><i style="background: #FF0000; width: 15px; height: 15px; display: inline-block;"></i> pH ≤ 4 (Highly Acidic)</div>
        <div><i style="background: #FFFF00; width: 15px; height: 15px; display: inline-block;"></i> 4 < pH < 6.5 (Moderately Acidic)</div>
        <div><i style="background: #00FF00; width: 15px; height: 15px; display: inline-block;"></i> 6.5 ≤ pH ≤ 8.5 (Acceptable)</div>
        <div><i style="background: #00BFFF; width: 15px; height: 15px; display: inline-block;"></i> 8.5 < pH < 10 (Moderately Alkaline)</div>
        <div><i style="background: #8A2BE2; width: 15px; height: 15px; display: inline-block;"></i> pH ≥ 10 (Highly Alkaline)</div>
        </div>
        '''
    elif parameter == 'DO':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; 
                    padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <h4>Dissolved Oxygen (DO) Levels</h4>
        <div><i style="background: #00FF00; width: 15px; height: 15px; display: inline-block;"></i> DO ≥ 5 mg/L (Acceptable)</div>
        <div><i style="background: #FF0000; width: 15px; height: 15px; display: inline-block;"></i> DO < 5 mg/L (Low Oxygen)</div>
        <div><i style="background: gray; width: 15px; height: 15px; display: inline-block;"></i> No Data</div>
        </div>
        '''
    elif parameter == 'BOD':
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; 
                    padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <h4>Biochemical Oxygen Demand (BOD) Levels</h4>
        <div><i style="background: #00FF00; width: 15px; height: 15px; display: inline-block;"></i> BOD ≤ 3 mg/L (Acceptable)</div>
        <div><i style="background: #FF0000; width: 15px; height: 15px; display: inline-block;"></i> BOD > 3 mg/L (High BOD)</div>
        <div><i style="background: gray; width: 15px; height: 15px; display: inline-block;"></i> No Data</div>
        </div>
        '''
    
    india_map.get_root().html.add_child(folium.Element(legend_html))

    return india_map

def create_parameter_year_selector_app():
    """Create an interactive application with year and parameter selector"""
    # Create output directories for maps
    if not os.path.exists('maps'):
        os.makedirs('maps')
    if not os.path.exists('maps/pH'):
        os.makedirs('maps/pH')
    if not os.path.exists('maps/DO'):
        os.makedirs('maps/DO')
    if not os.path.exists('maps/BOD'):
        os.makedirs('maps/BOD')
    
    # Create maps for each year and parameter and save them
    parameters = ['pH', 'DO', 'BOD']
    for year in range(2012, 2024):
        data = load_data_for_year(year)
        if data is not None:
            for param in parameters:
                map_obj = create_river_map(data, year, parameter=param)
                map_obj.save(f'maps/{param}/india_river_{param.lower()}_map_{year}.html')
                print(f"Generated {param} map for {year}")
    
    # Create HTML file with year and parameter selectors
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>India River Water Quality Map</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }
            .container {
                display: flex;
                flex-direction: column;
                height: 100vh;
            }
            .header {
                background-color: #f0f0f0;
                padding: 10px;
                text-align: center;
            }
            .content {
                display: flex;
                flex: 1;
            }
            .map-container {
                flex: 1;
                position: relative;
            }
            iframe {
                width: 100%;
                height: 100%;
                border: none;
            }
            .selectors {
                padding: 20px;
                display: flex;
                justify-content: center;
                align-items: center;
                gap: 20px;
                flex-wrap: wrap;
            }
            .parameter-selector {
                display: flex;
                flex-direction: column;
                align-items: center;
            }
            .year-selector {
                display: flex;
                justify-content: center;
                gap: 10px;
                flex-wrap: wrap;
            }
            select {
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ccc;
                font-size: 16px;
                margin-top: 5px;
            }
            .year-btn {
                padding: 10px 15px;
                background-color: #e0e0e0;
                border: 1px solid #ccc;
                cursor: pointer;
                border-radius: 4px;
            }
            .year-btn:hover {
                background-color: #d0d0d0;
            }
            .year-btn.active {
                background-color: #0078D7;
                color: white;
            }
            label {
                font-weight: bold;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>India River Water Quality Map</h1>
            </div>
            <div class="selectors">
                <div class="parameter-selector">
                    <label for="parameter">Select Parameter:</label>
                    <select id="parameter" onchange="changeParameter()">
                        <option value="pH">pH</option>
                        <option value="DO">Dissolved Oxygen (DO)</option>
                        <option value="BOD">Biochemical Oxygen Demand (BOD)</option>
                    </select>
                </div>
                <div class="year-selector">
    """
    
    # Add year buttons
    for year in range(2012, 2024):
        if os.path.exists(f'maps/pH/india_river_ph_map_{year}.html'):
            active = 'active' if year == 2012 else ''
            html_content += f'<button class="year-btn {active}" onclick="changeYear({year})">{year}</button>\n'
    
    html_content += """
                </div>
            </div>
            <div class="content">
                <div class="map-container">
                    <iframe id="map-frame" src="maps/pH/india_river_ph_map_2012.html"></iframe>
                </div>
            </div>
        </div>
        <script>
            let currentYear = 2012;
            let currentParameter = 'pH';
            
            function updateMap() {
                // Update iframe source based on current selections
                const paramFolder = currentParameter;
                const paramLower = currentParameter.toLowerCase();
                document.getElementById('map-frame').src = `maps/${paramFolder}/india_river_${paramLower}_map_${currentYear}.html`;
            }
            
            function changeParameter() {
                currentParameter = document.getElementById('parameter').value;
                updateMap();
            }
            
            function changeYear(year) {
                currentYear = year;
                updateMap();
                
                // Update active button
                const buttons = document.querySelectorAll('.year-btn');
                buttons.forEach(btn => {
                    if (btn.textContent == year) {
                        btn.classList.add('active');
                    } else {
                        btn.classList.remove('active');
                    }
                });
            }
        </script>
    </body>
    </html>
    """
    
    # Write HTML to file
    with open('india_river_water_quality_map_selector.html', 'w') as f:
        f.write(html_content)
    
    print("Parameter and year selector app created! Open 'india_river_water_quality_map_selector.html' in your browser.")

# Run the parameter and year selector app
create_parameter_year_selector_app()

Generated pH map for 2012
Generated DO map for 2012
Generated BOD map for 2012
Generated pH map for 2013
Generated DO map for 2013
Generated BOD map for 2013
Generated pH map for 2014
Generated DO map for 2014
Generated BOD map for 2014
Generated pH map for 2015
Generated DO map for 2015
Generated BOD map for 2015
Generated pH map for 2016
Generated DO map for 2016
Generated BOD map for 2016
Generated pH map for 2017
Generated DO map for 2017
Generated BOD map for 2017
Generated pH map for 2018
Generated DO map for 2018
Generated BOD map for 2018
Generated pH map for 2019
Generated DO map for 2019
Generated BOD map for 2019
Generated pH map for 2020
Generated DO map for 2020
Generated BOD map for 2020
Generated pH map for 2021
Generated DO map for 2021
Generated BOD map for 2021
Generated pH map for 2022
Generated DO map for 2022
Generated BOD map for 2022
Generated pH map for 2023
Generated DO map for 2023
Generated BOD map for 2023
Parameter and year selector app created! Open 'india